## Swedcham Debit Note Automation

1. Read American Credit Card Invoice
2. Gather all needed pieces of information
3. Create a new Word File with the current Debit Note
4. Generate a PDF from the Word File
5. Send a mail to Swedcham

#### 1. Read American Credit Card Invoice

In [1]:
# Import Libraries

# Common Libraries
import datetime
import os
import re
import datetime
import locale
import json

# Word/PDF Libraries
import PyPDF2
from docxtpl import DocxTemplate

# Email libraries
import smtplib, ssl
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# Cryptography Library
from cryptography.fernet import Fernet


In [20]:
# Load Script Parameters

# Code References
    # https://www.scaler.com/topics/how-to-open-json-file-in-python/
    # https://bobbyhadz.com/blog/python-assign-dictionary-value-to-variable#assign-dictionary-key-value-pairs-to-variables-in-python

# Open json file with the parameters
with open("Swedcham-Debit-Note-Automation-Params.json", "r") as file:
    jsonData = json.load(file)
# print("Datatype of variable: ", type(jsonData))

# Initiate variables
locals().update(jsonData['crypto'])
locals().update(jsonData['credit_card'])
locals().update(jsonData['debit_note'])
locals().update(jsonData['mail'])

# Decode passwords
key = bytes(pwd_key, 'utf-8')
fernet = Fernet(key)

cc_pass = fernet.decrypt(bytes(cc_pass, 'utf-8')).decode()
mail_password = fernet.decrypt(bytes(mail_password, 'utf-8')).decode()

print(nd_mail_body)

Olá Elaine,

Em anexo a Nota de Débito do Mailchimp deste mês.

Oscar


In [6]:
# Get last invoice file name

# Get current year
today = datetime.date.today()
year = str(today.year)
# print(year)

path = cc_folder + year + '\\'

file_list = [entry.name for entry in os.scandir(path) if entry.name[0:4] == year]
filename = file_list[len(file_list)-1]

print(path)
print(filename)

D:\Primeway\Users Pw\Oscar.Carvalho\Contas\Cartões de Crédito\American\2023\
2023-08-Extrato-American-Bradesco.pdf


In [7]:
# Open PDF file and get Mailchimp line

with open(path + filename, 'rb') as obj:
   pdfReader = PyPDF2.PdfReader(obj)
   if pdfReader.is_encrypted:
      pdfReader.decrypt(cc_pass)
   page_txt = pdfReader.pages[0].extract_text()

pattern = '[0-9]+/[0-9]+/[0-9]+.+' + 'MAILCHIMP.+' + '\\n'
mailchimp_line = re.search(pattern, page_txt)
if (not mailchimp_line == None):
   mailchimp_line = mailchimp_line.group().replace(',', '.')
   print(mailchimp_line)
else:
   print('Mailchimp line not found!')

19/07/2023 MAILCHIMP 55.00 USD 55.00 5.0500 277.75



#### 2. Gather all needed pieces of information

In [8]:
# Get most of the context variables from CC PDF

locale.setlocale(locale.LC_ALL, 'pt_BR')

items = re.split('[\s|\\n]', mailchimp_line)
nd_date_of_charge = items[0]
nd_value_usd      = items[4]
nd_value_brlusd   = items[5]
nd_value_brl      = items[6]

# Calc totals
nd_amount = round(float(nd_value_usd) * float(nd_value_brlusd) * (1 + nd_IOF) * (1 + nd_comission), 2)
nd_amount = str(nd_amount).replace('.', ',')

# Format values
nd_value_usd    = str(nd_value_usd).replace('.', ',')
nd_value_brlusd = str(round(float(nd_value_brlusd), 2)).replace('.', ',')

nd_description = nd_desc_template.format(nd_value_usd, nd_value_brlusd)
print(nd_description)

# Get current date
nd_current_date = datetime.date.today().strftime("%d de %B de %Y")
print(nd_current_date)


Plano Mensal MAILCHIMP - US$ 55,00 (cotação R$ 5,05 + 6,38% IOF + Comissão 10%)
11 de agosto de 2023


In [9]:
# Get due date

# Get date object from date of charge
date_obj = datetime.datetime.strptime(nd_date_of_charge, '%d/%m/%Y')
print(date_obj)
# # Calc due date
nd_due_date = date_obj.replace(day=1) + datetime.timedelta(days=31*2)
nd_due_date = nd_due_date - datetime.timedelta(days=nd_due_date.day)

# Format Due Date
nd_due_date = nd_due_date.strftime("%d/%m/%Y")
print(nd_due_date)


2023-07-19 00:00:00
31/08/2023


In [12]:
# Get sequence of the Nota de Débito

nd_sequence = ''

# Get filename
file_list = [entry.name for entry in os.scandir(nd_folder) if entry.name[0:8] == 'Primeway' and entry.name[-3:] == 'pdf']
filename = file_list[len(file_list)-1]
print(filename)

# Calc new sequence
pattern = '0[0-9]+(?=\s)'
last_sequence = re.search(pattern, filename)

nd_sequence = int(last_sequence.group()) + 1
nd_sequence = str(nd_sequence).zfill(4)
print(nd_sequence)

Primeway - Nota de Débito 0032 - 2023-07.pdf
0033


In [13]:
# Prepare context

context = { 
   'nd_current_date'   : nd_current_date,
   'nd_sequence'       : nd_sequence,
   'nd_recipient'      : mail_recipients[0],
   'nd_description'    : nd_description,
   'nd_date_of_charge' : nd_date_of_charge,
   'nd_amount'         : nd_amount,
   'nd_due_date'       : nd_due_date
}
pretty_context = json.dumps(context, indent=4)

print(pretty_context)

{
    "nd_current_date": "11 de agosto de 2023",
    "nd_sequence": "0033",
    "nd_recipient": "oscar@primeway.com.br",
    "nd_description": "Plano Mensal MAILCHIMP - US$ 55,00 (cota\u00e7\u00e3o R$ 5,05 + 6,38% IOF + Comiss\u00e3o 10%)",
    "nd_date_of_charge": "19/07/2023",
    "nd_amount": "325,02",
    "nd_due_date": "31/08/2023"
}


#### 3. Create a new Word File from the current Debit Note

In [23]:
# Open template and render
doc = DocxTemplate(nd_folder + nd_template)
doc.render(context)

# Save a new Nota de Débito
nd_word_filename = nd_filename.format(nd_sequence, nd_due_date[-4:], nd_due_date[3:5])
doc.save(nd_folder + nd_word_filename)

print(nd_word_filename)

TypeError: 'SSLContext' object is not iterable

#### 4. Generate a PDF from the Word File

In [17]:
# Create Word Application
from win32com import client
word_app = client.Dispatch("Word.Application")

# Generate PDF from Word
doc = word_app.Documents.Open(nd_folder + nd_word_filename)

nd_pdf_filename = nd_word_filename.replace('.docx', '.pdf')
doc.SaveAs(nd_folder + nd_pdf_filename, FileFormat=17)

# Close application
doc.Close()
word_app.Quit()
# doc.ActiveDocument.Save
# doc.ActiveDocument.Close()

print(nd_pdf_filename)

Primeway - Nota de Débito 0033 - 2023-08.pdf


#### 5. Send a mail with attached Debit Note to Swedcham

In [18]:
# Helper functions

# Attach files as MIMEApplication to the email
def attach_file_to_email(email_message, folder, filename):
    # Open the attachment file for reading in binary mode, and make it a MIMEApplication class
    with open(folder + filename, "rb") as f:
        file_attachment = MIMEApplication(f.read())
        # Add header/name to the attachments
        # Source: https://docs.python.org/3/library/email.message.html
        file_attachment.add_header('Content-Disposition', 'attachment', filename=('iso-8859-1', '', filename))

    # Attach the file to the message
    email_message.attach(file_attachment)

In [21]:
# Code References
   # https://www.youtube.com/watch?v=g_j6ILT-X0k&ab_channel=ThePyCoach
   # https://www.youtube.com/watch?v=NvtjLXSY-hE&ab_channel=LianneandJustin

# Google Resources
   # https://knowledge.workspace.google.com/kb/how-to-generate-an-app-passwords-000009237
   # https://app.s.center/hc/en-us/articles/1500012778262-Cannot-sign-in-to-Gmail-Invalid-email-address-or-password

# Mount the mail message
email_message = MIMEMultipart()
email_message['From'] = mail_sender
email_message['To'] = ', '.join(mail_recipients)
email_message['Subject'] = nd_mail_subject

# Attach mail body
email_message.attach(MIMEText(nd_mail_body, "plain"),)

# Attach Debit Note file
attach_file_to_email(email_message, nd_folder, nd_pdf_filename)

email_string = email_message.as_string()

# Add SSL (layer of security)
context = ssl.create_default_context()

# Log in and send the email
with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
    smtp.login(mail_sender, mail_password)
    smtp_return = smtp.sendmail(mail_sender, mail_recipients, email_string)
    print(smtp_success)

Email enviado com sucesso!
